In [83]:
# Format file 'Crime_Statistics_-_Incidents_and_rates_by_offence_category.csv'
import pandas as pd
import numpy as np

crimeStats = pd.read_csv('data/Crime_Statistics_-_Incidents_and_rates_by_offence_category.csv')

# Extract first geography string
geo = crimeStats.GEOGRAPHY.str.split(',').values
crimeStats['GEOGRAPHY2'] = [row[0] for row in geo]

# Format Year
from datetime import datetime
crimeStats['year'] = [datetime.strptime(y, '%m/%d/%Y %I:%M:%S %p').year for y in crimeStats.YEAR]
crimeStats.set_index(['GEOGRAPHY', 'year'], inplace=True)

rates = crimeStats.pivot_table(values='RATES', index=crimeStats.index, columns='VIOLATIONS', dropna=False)
ratesTransp = rates.values.transpose()
rateKeys = rates.keys()
newCrimeStats = crimeStats.pivot_table(values='INCIDENTS', index=crimeStats.index, columns='VIOLATIONS', dropna=False)

for i, key in enumerate(rateKeys):
    newCrimeStats['Rate ' + key] = ratesTransp[i]

idx = newCrimeStats.index.values
idx = np.array([np.array(row) for row in idx])
idx = idx.transpose()

newCrimeStats['geography'] = idx[0]
newCrimeStats['year'] = idx[1]
newCrimeStats.to_csv('data/proc/crime_stats_by_offence_category.csv', index=False)

In [108]:
counties = pd.read_csv('data/proc/ns_counties.csv')
newCrimeStats['CO_CODE'] = ['' for row in newCrimeStats.values]
geo = newCrimeStats.geography.str.split(',').values
line = -1

for i, row in newCrimeStats.iterrows():
    line += 1
    for j, county in counties.iterrows():
        name = county['CO_DESC'].lower()
        code = county['CO_CODE']

        if name in geo[line][0].lower():
            newCrimeStats.loc[i, 'CO_CODE'] = code